# This file currently exist to test the BGG API. The code is not yet in any particular order

Eventual information I hope to obtain:
 * Take about 100 comments from each to find what people do or don't like about certain games
   * See if that has any relation to the game's categories or mechanism
   * (Not sure if possible) See how these people have rated other games, find similar tastes
 * Take the score of a game and see how it relates to player count, age, and mechanism
   * Are some mechanisms consistently more popular? Are some more divisive?
   * Are newer games rated more highly? Have games gotten better?

# Setup

In [174]:
import requests
from lxml import etree
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import time
import sqlite3

In [223]:
con = sqlite3.connect('bgg.db')
cur = con.cursor()

In [175]:
def get_user_ratings(username):
    error_message = ['\n\tYour request for this collection has been accepted and will be processed.  Please try again later for access.\n']
    
    got_data = False
    # Make API call to get XML data
    while not got_data:
        params = {'username' : username, 'type': 'boardgame', 'rated' : '1', 'brief' : '1', 'stats' : '1'}
        response = requests.get('https://www.boardgamegeek.com/xmlapi2/collection?', params)
        s = BeautifulSoup(response.text, "xml")
        
        try:
            s.find('message').contents
        except AttributeError:
            got_data = True
        finally:
            time.sleep(1)
    
    # Initialize dataframe
    rating_df = pd.DataFrame(columns = ['id', 'rating'])

    # Loop through and output ratings to a dataframe
    for i in s.find_all('item'):
        game_id = i['objectid']
        
        # Make a soup out of the item info to extract rating
        inner_soup = BeautifulSoup(str(i))
        rv = inner_soup.find('stats')
        inner_soup2 = BeautifulSoup(str(rv))
        rating = inner_soup2.find('rating')
        rating = float(rating['value'])
        
        # Append rating to df
        temp_df = pd.DataFrame([[game_id,rating]], columns = ['id', 'rating'])
        rating_df = rating_df.append(temp_df)
    
    rating_df['index'] = username
    rating_df.rename({'index' :'username'}, inplace=True)
    
    return rating_df

In [176]:
con = sqlite3.connect('bgg.db')
cur = con.cursor()

In [177]:
ERROR_MESSAGE = error_message = ['\n\tYour request for this collection has been accepted and will be processed.  Please try again later for access.\n']

# Get Player Counts

In [183]:
# Get list of top 1000 games from SQL table
game_li = []

results=cur.execute("select id FROM TOP_BGG_GAMES")
results=results.fetchall()

for num in results:
    game_li.append(num[0])

In [211]:
# For each game, get the min and max player count:
min_max_player_li = []

i = 0
while i < len(game_li):
    game_id = game_li[i]
    params = {'id' : game_id,'stats' :'1'}
    response = requests.get('https://www.boardgamegeek.com/xmlapi2/thing?', params)
    soup = BeautifulSoup(response.text, "xml")

    try: 
        min_play = soup.find('minplayers')['value']
        max_play = soup.find('maxplayers')['value']
        i += 1
    except TypeError:
        # If it fails, we need to wait so we can use the API gain
        time.sleep(15)
    
    min_max_player_li.append([game_id, min_play, max_play])

In [225]:
# Create table
cur.execute(''' CREATE TABLE BGG_PLAYER_COUNTS
               (id text, min_players text, max_players text)''')

In [231]:
# Remove duplicates from API
player_li_df = pd.DataFrame(min_max_player_li, columns = ['GameId','MinPlayers','MaxPlayers'])
player_li_df.drop_duplicates(subset = 'GameId', keep='first', inplace=True)
player_li_df.reset_index(inplace=True)
player_li_df.shape

(1000, 4)

In [233]:
# Convert dataframe to tuple
df = player_li_df
tuple_li = []
for i in range(len(df)):
    tuple_ = str(df.loc[i, 'GameId']), df.loc[i, 'MinPlayers'], str(df.loc[i, 'MaxPlayers'])
    tuple_li.append(tuple_)

In [234]:
# Enter tuple into SQL table
cur.executemany("insert into BGG_PLAYER_COUNTS values (?, ?, ?)", tuple_li)

In [237]:
con.commit()

# Obtain game categories and mechanism

In [250]:
category_li, mechanic_li = [],[]
loop_len = len(game_li)

i = 0
while i < loop_len:
    game_id = game_li[i]
    params = {'id' : game_id}
    response = requests.get('https://www.boardgamegeek.com/xmlapi2/thing?', params)
    soup = BeautifulSoup(response.text, 'xml')
    info_ = soup.find_all('link')
    
    try:
        for x in info_:
            if x['type'] == 'boardgamecategory':
                category_li.append((game_id, x['value']))
            elif x['type'] == 'boardgamemechanic':
                mechanic_li.append((game_id, x['value']))
        i += 1 # increment the loop
    except TypeError:  # occurs when API times out
        time.sleep(10) # wait ten seconds so we can run the API gain

In [254]:
# Create table
cur.execute(''' CREATE TABLE BGG_CATEGORIES
               (id text, category text);
               ''')

cur.execute('''CREATE TABLE BGG_MECHANICS
               (id text, mechanic text);''')

In [257]:
# Enter tuple into SQL table
cur.executemany("insert into BGG_CATEGORIES values (?, ?)", category_li)
cur.executemany("insert into BGG_MECHANICS values (?, ?)", mechanic_li)

In [258]:
con.commit()

In [294]:
params = {'id' : 174430,'stats' :'1'}
response = requests.get('https://www.boardgamegeek.com/xmlapi2/thing?', params)
s = BeautifulSoup(response.text, "xml")
all_stats = s.find('ratings')
# s2 = BeautifulSoup(str(all_stats), 'xml')
# s2.find('ratings')
# s2['usersrated']
#all_stats.find('usersrated')['value']
all_stats

<ratings>
<usersrated value="45304"/>
<average value="8.76798"/>
<bayesaverage value="8.53089"/>
<ranks>
<rank bayesaverage="8.53089" friendlyname="Board Game Rank" id="1" name="boardgame" type="subtype" value="1"/>
<rank bayesaverage="8.51374" friendlyname="Thematic Rank" id="5496" name="thematic" type="family" value="1"/>
<rank bayesaverage="8.5007" friendlyname="Strategy Game Rank" id="5497" name="strategygames" type="family" value="1"/>
</ranks>
<stddev value="1.63005"/>
<median value="0"/>
<owned value="73432"/>
<trading value="557"/>
<wanting value="1383"/>
<wishing value="17012"/>
<numcomments value="8156"/>
<numweights value="1960"/>
<averageweight value="3.8673"/>
</ratings>

In [297]:
avg_rating = all_stats.find('average')['value']
avg_bayes_rating = all_stats.find('bayesaverage')['value']
std = all_stats.find('stddev')['value']
ratings = all_stats.find('usersrated')['value']
weight = all_stats.find('averageweight')['value']

In [298]:
avg_rating, avg_bayes_rating, std, ratings, weight

('8.76798', '8.53089', '1.63005', '45304', '3.8673')

# Scratch Code

In [213]:
# Find the votes with respect to the number of players
results = soup.find_all('results', {'numplayers':'1'})

# general text for vote counts
soup2 = BeautifulSoup(str(results[0]), 'xml')
print(soup2)

# Specific value of the recommendation for 1 player
# Side note, this is a troll vote as Catan requires at least 3 player
soup2.find('result', {'value' :'Best'})['numvotes']

<?xml version="1.0" encoding="utf-8"?>
<results numplayers="1">
<result numvotes="1" value="Best"/>
<result numvotes="3" value="Recommended"/>
<result numvotes="1101" value="Not Recommended"/>
</results>


'1'

In [190]:
# I can also obtain the stats for a game
params = {'id' : '13', 'ratingcomments' : '1', 'comments' : '1', 'pagesize' : '100', 'page' :'1', 'stats' :'1'}
response = requests.get('https://www.boardgamegeek.com/xmlapi2/thing?', params)

<?xml version="1.0" encoding="utf-8"?>
<items termsofuse="https://boardgamegeek.com/xmlapi/termsofuse"><item id="13" type="boardgame">
<thumbnail>https://cf.geekdo-images.com/W3Bsga_uLP9kO91gZ7H8yw__thumb/img/8a9HeqFydO7Uun_le9bXWPnidcA=/fit-in/200x150/filters:strip_icc()/pic2419375.jpg</thumbnail>
<image>https://cf.geekdo-images.com/W3Bsga_uLP9kO91gZ7H8yw__original/img/xV7oisd3RQ8R-k18cdWAYthHXsA=/0x0/filters:format(jpeg)/pic2419375.jpg</image>
<name sortindex="1" type="primary" value="Catan"/>
<name sortindex="1" type="alternate" value="CATAN"/>
<name sortindex="1" type="alternate" value="Catan (Колонизаторы)"/>
<name sortindex="1" type="alternate" value="Catan telepesei"/>
<name sortindex="1" type="alternate" value="Catan: Das Spiel"/>
<name sortindex="1" type="alternate" value="Catan: Die Bordspel"/>
<name sortindex="1" type="alternate" value="Catan: El Juego"/>
<name sortindex="1" type="alternate" value="Catan: Gra planszowa"/>
<name sortindex="1" type="alternate" value="Catan: Il

In [29]:
# We look at certain users from the comments and see what other games they might like

params = {'name' : 'agentzen', 'hot' : '1', 'top' : '1', 'domain' : 'boardgame',}
response = requests.get('https://www.boardgamegeek.com/xmlapi2/user?', params)
s = BeautifulSoup(response.text, "xml")
s

<?xml version="1.0" encoding="utf-8"?>
<user id="3211" name="agentzen" termsofuse="https://boardgamegeek.com/xmlapi/termsofuse">
<firstname value="Aaron"/> <lastname value="Cappocchi"/> <avatarlink value="https://cf.geekdo-static.com/avatars/avatar_id47464.jpg"/> <yearregistered value="2003"/> <lastlogin value="2021-08-24"/> <stateorprovince value="CA"/> <country value="United States"/> <webaddress value=""/> <xboxaccount value=""/> <wiiaccount value=""/> <psnaccount value="sonar_system"/> <battlenetaccount value=""/> <steamaccount value="aaronsteamz"/> <traderating value="100"/> <marketrating value="8"/>
<top domain="boardgame">
<item id="17392" name="Here I Stand" rank="1" type="thing"/>
<item id="8045" name="Princes of the Renaissance" rank="2" type="thing"/>
<item id="2507" name="Liberté" rank="3" type="thing"/>
<item id="22038" name="Warrior Knights" rank="4" type="thing"/>
<item id="12333" name="Twilight Struggle" rank="5" type="thing"/>
<item id="5404" name="Amun-Re" rank="6" ty

In [47]:
# We look at certain users from the comments and see what other games they might like
params = {'username' : 'agentzen', 'type': 'boardgame', 'rated' : '1', 'brief' : '1', 'stats' : '1'}
response = requests.get('https://www.boardgamegeek.com/xmlapi2/collection?', params)
s = BeautifulSoup(response.text, "xml")
s

<?xml version="1.0" encoding="utf-8"?>
<items pubdate="Tue, 24 Aug 2021 13:44:07 +0000" termsofuse="https://boardgamegeek.com/xmlapi/termsofuse" totalitems="844">
<item collid="5730286" objectid="4550" objecttype="thing" subtype="boardgame">
<name sortindex="1">1000 Blank White Cards</name>
<stats maxplayers="99" maxplaytime="60" minplayers="3" minplaytime="60" numowned="499" playingtime="60">
<rating value="4"> <average value="6.66249"/>
<bayesaverage value="5.75539"/></rating>
</stats> <status fortrade="0" lastmodified="2007-11-20 18:22:40" own="0" preordered="0" prevowned="0" want="0" wanttobuy="0" wanttoplay="0" wishlist="0"/>
</item>
<item collid="4976667" objectid="22278" objecttype="thing" subtype="boardgame">
<name sortindex="1">12 Thieves</name>
<stats maxplayers="4" maxplaytime="45" minplayers="2" minplaytime="45" numowned="2052" playingtime="45">
<rating value="4.5"> <average value="6.44087"/>
<bayesaverage value="5.94164"/></rating>
</stats> <status fortrade="0" lastmodifie

In [115]:
# Comments
comment_li = []

# Obtaining ten pages of user comments/ratings
for i in range(1,11):
    params = {'id' : '13', 'ratingcomments' : '1', 'comments' : '1', 'pagesize' : '100', 'page':[3,4], 'stats' :str(i)}
    response = requests.get('https://www.boardgamegeek.com/xmlapi2/thing?', params)
    comments = soup.find_all("comment")
    for com in comments:
        comment_li.append(com)

In [140]:
test_df = get_user_ratings('agent_t_bib')
test_df

,id,rating,index
0,63706,6.0,agent_t_bib
0,22278,7.0,agent_t_bib
0,89972,6.0,agent_t_bib
0,82955,7.0,agent_t_bib
0,3377,6.0,agent_t_bib
...,...,...,...
0,8593,7.0,agent_t_bib
0,2243,4.0,agent_t_bib
0,12761,7.0,agent_t_bib
0,28246,7.0,agent_t_bib
